# Issue 1

Issue https://github.com/tensorflow/tensorflow/issues/27696

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging

logging.set_verbosity(logging.INFO)
# Define the estimator's input_fn
STEPS_PER_EPOCH = 5
#BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 5


def input_fn():
    datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
    
        return image, label[..., tf.newaxis]

    train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_data.repeat()

# Define train & eval specs
train_spec = tf.estimator.TrainSpec(input_fn=input_fn,
                                    max_steps=STEPS_PER_EPOCH * NUM_EPOCHS)
eval_spec = tf.estimator.EvalSpec(input_fn=input_fn,
                                  steps=STEPS_PER_EPOCH)

def make_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(0.02),
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model = make_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#####
## works
#strategy=None 
## crash
strategy = tf.distribute.MirroredStrategy()

# config tf.estimator to use a give strategy
training_config = tf.estimator.RunConfig(train_distribute=strategy)
#####

estimator = tf.keras.estimator.model_to_estimator(
    keras_model = model,
    config=training_config
)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

W0410 10:15:55.945059 4556207552 cross_device_ops.py:1111] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.
I0410 10:15:55.945975 4556207552 run_config.py:532] Initializing RunConfig with distribution strategies.
I0410 10:15:55.946563 4556207552 estimator_training.py:167] Not using Distribute Coordinator.
W0410 10:15:55.948405 4556207552 estimator.py:1799] Using temporary folder as model directory: /var/folders/l7/00kxfwvs0vbbqxtrp3rpf3yh0000gn/T/tmpyvqcdnlw
I0410 10:15:55.949310 4556207552 keras.py:464] Using the Keras model provided.
I0410 10:15:56.822856 4556207552 estimator.py:202] Using config: {'_model_dir': '/var/folders/l7/00kxfwvs0vbbqxtrp3rpf3yh0000gn/T/tmpyvqcdnlw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every

ValueError: Only TensorFlow native optimizers are supported with DistributionStrategy.

# Issue 2

Issue https://github.com/tensorflow/tensorflow/issues/27581

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging

logging.set_verbosity(logging.INFO)
# Define the estimator's input_fn
STEPS_PER_EPOCH = 5
#BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 5


def input_fn():
    datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
    
        return image, label[..., tf.newaxis]

    train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_data.repeat()


def make_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(0.02),
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model = make_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

training_dataset=input_fn()

print("train")
model.fit(training_dataset,
          steps_per_epoch=5,
          epochs=10,
          verbose = 1)

print("evaluate")
model.evaluate(training_dataset,
              steps=1)

print("predict on batch")
model.predict_on_batch(training_dataset)

I0417 11:31:40.511445 4620772800 dataset_builder.py:157] Overwrite dataset info from restored data version.
I0417 11:31:40.516194 4620772800 dataset_builder.py:193] Reusing dataset mnist (/Users/tarrade/tensorflow_datasets/mnist/1.0.0)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                346176    
_________________________________________________________________
batch_normalization_v2 (Batc (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

RuntimeError: dataset.make_initializable_iterator is not supported when eager execution is enabled.

# Issue 3

In [34]:
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging
from tensorboard.backend.event_processing import event_accumulator
import numpy as np

logging.set_verbosity(logging.INFO)
# Define the estimator's input_fn
STEPS_PER_EPOCH = 5
#BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 1000


def input_fn():
    datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
    
        return image, label[..., tf.newaxis]

    train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_data.repeat()

def make_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(0.02),
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model = make_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# config tf.estimator to use a give strategy
training_config = tf.estimator.RunConfig(model_dir='/tmp/test2',
                                         save_summary_steps=10,  # save summary every n steps
                                         save_checkpoints_steps=10,  # save model every iteration (needed for eval)
                                         # save_checkpoints_secs=10,
                                         keep_checkpoint_max=3,  # keep last n models
                                         log_step_count_steps=50)

# Define train & eval specs
train_spec = tf.estimator.TrainSpec(input_fn=input_fn,
                                    max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=input_fn,
                                  steps=1)

estimator = tf.keras.estimator.model_to_estimator(
    keras_model = model,
    config=training_config
)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

print('train the model to estimator')
#estimator.train(input_fn=input_fn,
#               steps=1000)

def load_data_tensorboard(path):
    event_acc = event_accumulator.EventAccumulator(path)
    event_acc.Reload()
    data = {}

    for tag in sorted(event_acc.Tags()["scalars"]):
        x, y = [], []
        for scalar_event in event_acc.Scalars(tag):
            x.append(scalar_event.step)
            y.append(scalar_event.value)
        data[tag] = (np.asarray(x), np.asarray(y))
    return data

print('let inspect events.out.tfevents.* files')
#print(' ')
#print('training:')
#history_train=load_data_tensorboard('/tmp/test2/')
#print(history_train.keys())
#print(history_train['loss_1'])
#print(' ')
#print('evaluation:')
#history_eval=load_data_tensorboard('/tmp/test2/eval')
#print(history_eval.keys())
#print(history_eval['loss'][1])
#print(history_eval['accuracy'][1])
#print()
#print(' ')

I0418 10:32:48.354762 4620772800 keras.py:464] Using the Keras model provided.
I0418 10:32:49.495244 4620772800 estimator.py:202] Using config: {'_model_dir': '/tmp/test2', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb33e095c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I0418 10:32:49.496958 4620772800 estimator_training.py:186] Not using Distrib

I0418 10:33:01.104421 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 80 into /tmp/test2/model.ckpt.
I0418 10:33:01.210136 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:01.460709 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 90 into /tmp/test2/model.ckpt.
I0418 10:33:01.568709 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:01.823717 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 100 into /tmp/test2/model.ckpt.
I0418 10:33:01.928065 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:01.957066 4620772800 basic_session_run_hooks.py:680] global_step/sec: 24.6102
I0418 10:33:01.958796 4620772800 basic_session_run_hooks.py:247] loss = 0.24310794, step = 100 (2.032 sec)
I0418 10:33:02.176247 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 110 into 

I0418 10:33:12.673862 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 380 into /tmp/test2/model.ckpt.
I0418 10:33:12.791600 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:13.058331 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 390 into /tmp/test2/model.ckpt.
I0418 10:33:13.173775 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:13.429002 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 400 into /tmp/test2/model.ckpt.
I0418 10:33:13.545475 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:13.573698 4620772800 basic_session_run_hooks.py:680] global_step/sec: 25.4304
I0418 10:33:13.575514 4620772800 basic_session_run_hooks.py:247] loss = 0.14879647, step = 400 (1.966 sec)
I0418 10:33:13.822032 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 410 int

I0418 10:33:24.244512 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 680 into /tmp/test2/model.ckpt.
I0418 10:33:24.355572 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:24.636538 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 690 into /tmp/test2/model.ckpt.
I0418 10:33:24.755194 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:25.015833 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 700 into /tmp/test2/model.ckpt.
I0418 10:33:25.124302 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:25.154242 4620772800 basic_session_run_hooks.py:680] global_step/sec: 26.099
I0418 10:33:25.155678 4620772800 basic_session_run_hooks.py:247] loss = 0.08401941, step = 700 (1.916 sec)
I0418 10:33:25.400709 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 710 into

I0418 10:33:35.949110 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 980 into /tmp/test2/model.ckpt.
I0418 10:33:36.058755 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:36.332733 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 990 into /tmp/test2/model.ckpt.
I0418 10:33:36.440008 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:36.748717 4620772800 basic_session_run_hooks.py:594] Saving checkpoints for 1000 into /tmp/test2/model.ckpt.
I0418 10:33:36.857003 4620772800 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
I0418 10:33:36.867471 4620772800 dataset_builder.py:157] Overwrite dataset info from restored data version.
I0418 10:33:36.870707 4620772800 dataset_builder.py:193] Reusing dataset mnist (/Users/tarrade/tensorflow_datasets/mnist/1.0.0)
I0418 10:33:37.105168 4620772800 estimator.py:1126] Cal

train the model to estimator
let inspect events.out.tfevents.* files


In [8]:
!ls -la /tmp/test2/

total 28488
drwxr-xr-x  16 tarrade  wheel      512 Apr 10 16:05 .
drwxrwxrwt  24 root     wheel      768 Apr 10 16:05 ..
-rw-r--r--   1 tarrade  wheel      177 Apr 10 16:05 checkpoint
drwxr-xr-x   3 tarrade  wheel       96 Apr 10 16:05 eval
-rw-r--r--   1 tarrade  wheel   814064 Apr 10 16:05 events.out.tfevents.1554905112.Fabien-Tarrades-MacBook-Pro.local
-rw-r--r--   1 tarrade  wheel   542088 Apr 10 16:05 graph.pbtxt
drwxr-xr-x   6 tarrade  wheel      192 Apr 10 16:05 keras
-rw-r--r--   1 tarrade  wheel  4167828 Apr 10 16:05 model.ckpt-1000.data-00000-of-00001
-rw-r--r--   1 tarrade  wheel     1168 Apr 10 16:05 model.ckpt-1000.index
-rw-r--r--   1 tarrade  wheel   230862 Apr 10 16:05 model.ckpt-1000.meta
-rw-r--r--   1 tarrade  wheel  4167828 Apr 10 16:05 model.ckpt-980.data-00000-of-00001
-rw-r--r--   1 tarrade  wheel     1168 Apr 10 16:05 model.ckpt-980.index
-rw-r--r--   1 tarrade  wheel   230862 Apr 10 16:05 model.ckpt-980.meta
-rw-r--r--   1 tarrade  wheel  4167828 Apr 10 16:05 m

In [9]:
%load_ext tensorboard.notebook
%tensorboard  --logdir   {'/tmp/test2'}

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


Reusing TensorBoard on port 6013 (pid 84571), started -1 day, 23:21:03 ago. (Use '!kill 84571' to kill it.)

# Issue 4

In [33]:
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import logging

logging.set_verbosity(logging.INFO)
# Define the estimator's input_fn
STEPS_PER_EPOCH = 5
#BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 5

def input_fn():
    datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
    
        return image, label[..., tf.newaxis]

    train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_data.repeat()


def make_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu',
                               kernel_regularizer=tf.keras.regularizers.l2(0.02),
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

tbCallBack=tf.keras.callbacks.TensorBoard(log_dir='/tmp/test3/',
                                          histogram_freq=1,
                                          write_graph=True)

model = make_model()

optimiser=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, epsilon=1e-07)

model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

training_dataset=input_fn()

print("train")
model.fit(training_dataset,
          steps_per_epoch=5,
          epochs=10,
          callbacks=[tbCallBack],
          verbose =1)

#print("evaluate")
#model.evaluate(training_dataset,
#              steps=1)

I0417 13:41:14.449864 4620772800 dataset_builder.py:157] Overwrite dataset info from restored data version.
I0417 13:41:14.452956 4620772800 dataset_builder.py:193] Reusing dataset mnist (/Users/tarrade/tensorflow_datasets/mnist/1.0.0)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 5408)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                346176    
_________________________________________________________________
batch_normalization_v2_5 (Ba (None, 64)                256       
_________________________________________________________________
dense_14 (Dense)             (None, 10)               

In [6]:
%load_ext tensorboard.notebook
%tensorboard  --logdir   {'/tmp/test3'}

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


Reusing TensorBoard on port 6006 (pid 17713), started -1 day, 23:03:03 ago. (Use '!kill 17713' to kill it.)

In [3]:
!ls -la /tmp/test3/

total 0
drwxr-xr-x   5 tarrade  wheel  160 Apr 10 22:03 .
drwxrwxrwt  12 root     wheel  384 Apr 10 22:03 ..
drwxr-xr-x   3 tarrade  wheel   96 Apr 10 22:03 plugins
drwxr-xr-x   3 tarrade  wheel   96 Apr 10 22:03 train
drwxr-xr-x   3 tarrade  wheel   96 Apr 10 22:03 validation


In [14]:
!ls -la ../../results/Models/Mnist/tf_1_12/estimator/v2/ckpt

total 48664
drwxr-xr-x  14 tarrade  staff      448 Apr 10 21:17 .
drwxr-xr-x   4 tarrade  staff      128 Apr 10 21:13 ..
-rw-r--r--   1 tarrade  staff      177 Apr 10 21:17 checkpoint
-rw-r--r--   1 tarrade  staff   333317 Apr 10 21:13 graph.pbtxt
drwxr-xr-x   6 tarrade  staff      192 Apr 10 21:13 keras
-rw-r--r--   1 tarrade  staff  8036500 Apr 10 21:17 model.ckpt-1000.data-00000-of-00001
-rw-r--r--   1 tarrade  staff      889 Apr 10 21:17 model.ckpt-1000.index
-rw-r--r--   1 tarrade  staff   143942 Apr 10 21:17 model.ckpt-1000.meta
-rw-r--r--   1 tarrade  staff  8036500 Apr 10 21:17 model.ckpt-980.data-00000-of-00001
-rw-r--r--   1 tarrade  staff      889 Apr 10 21:17 model.ckpt-980.index
-rw-r--r--   1 tarrade  staff   143942 Apr 10 21:17 model.ckpt-980.meta
-rw-r--r--   1 tarrade  staff  8036500 Apr 10 21:17 model.ckpt-990.data-00000-of-00001
-rw-r--r--   1 tarrade  staff      889 Apr 10 21:17 model.ckpt-990.index
-rw-r--r--   1 tarrade  staff   143942 Apr 10 21:17 model.ckpt-990.

In [1]:
import tensorflow as tf
tf.distribute.MirroredStrategy()

W0408 14:08:27.861193 4771767744 cross_device_ops.py:1111] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [8]:
import tensorflow as tf
from absl import logging
logging.set_verbosity(logging.DEBUG)
logging.debug('test')
tf.distribute.MirroredStrategy()

I0408 14:18:37.378367 4771767744 <ipython-input-8-2fd3015b66b4>:4] test
W0408 14:18:37.384752 4771767744 cross_device_ops.py:1111] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [1]:
from google.cloud import storage
import os
import pandas as pd

In [2]:
pd.__version__

'0.24.0'

In [3]:
#os.environ['HTTPS_PROXY'] = "https://proxy-url:Port"
#os.environ['REQUESTS_CA_BUNDLE'] = "C:/Users/path/to/certs"
#os.environ["PROJECT_ID"] = "project-id-34914"

In [4]:
project=os.environ['PROJECT_ID']

In [5]:
client = storage.Client(project=project)

C:\Program Files\Anaconda3\envs\mnist\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
for b in client.list_buckets():
    print(b.name)

In [7]:
from google.cloud import bigquery
client = bigquery.Client(project=project)

In [8]:
query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100 * SUM(IF(answer_count > 0, 1, 0)) / COUNT(*), 1) AS Percent_Questions_with_Answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2016
ORDER BY
  Year
"""
#df = client.query(query).to_dataframe()
#df.head()
query_job = client.query(query)
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((2009, 343139, 99.7), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2010, 693332, 99.1), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2011, 1198587, 97.2), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2012, 1642687, 94.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2013, 2056613, 91.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2014, 2160361, 88.5), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2015, 2214389, 86.4), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})


In [9]:
print("Keys: ", tuple(row.keys()))
print("Values: ", row.values())

Keys:  ('Year', 'Number_of_Questions', 'Percent_Questions_with_Answers')
Values:  (2015, 2214389, 86.4)


In [10]:
df = client.query(query).to_dataframe()
df

,Year,Number_of_Questions,Percent_Questions_with_Answers
0,2009,343139,99.7
1,2010,693332,99.1
2,2011,1198587,97.2
3,2012,1642687,94.6
4,2013,2056613,91.6
5,2014,2160361,88.5
6,2015,2214389,86.4
